In [1]:
from datetime import datetime
from typing import Annotated, Any

import pytz
from dateutil.parser import parse

from pydantic import BaseModel, AfterValidator, BeforeValidator

In [2]:
def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt
    
def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value


DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [3]:
class Model(BaseModel):
    dt: datetime

In [62]:
def dt_json_serialize(dt: datetime) -> str:
    print("serialized")
    return dt.strftime("%Y/%m/%d %I:%M %p (UTC)")

In [6]:
from pydantic import field_serializer

class Model(BaseModel):
    dt: DateTimeUTC
    
    @field_serializer("dt", when_used="json-unless-none")
    def serialize_dt_to_json(self, value: datetime) -> str:
        return dt_json_serialize(value)

In [7]:
m = Model(dt='2020/1/1 3pm')

In [8]:
m.model_dump()

{'dt': datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>)}

In [9]:
m.model_dump_json()

'{"dt":"2020/01/01 03:00 PM (UTC)"}'

In [11]:
from pydantic import PlainSerializer

In [63]:
DateTimeUTC = Annotated[
    datetime,
    BeforeValidator(parse_datetime),
    AfterValidator(make_utc),
    PlainSerializer(
        dt_json_serialize, 
        when_used='json-unless-none'
    )
]

In [65]:
class Model(BaseModel):
    dt: DateTimeUTC

In [67]:
m = Model(dt="2020/1/1")

In [68]:
m.model_dump()

{'dt': datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>)}

In [69]:
m.model_dump_json()

serialized


'{"dt":"2020/01/01 12:00 AM (UTC)"}'